# Homework 5 - The eternal significance of publications and citations!

This project was carried out by Group 19 of Algorithmic Methods for Data Mining, consisting of:

| NAME and SURNAME | EMAIL |
| --- | --- |
| Pasquale Luca Tommasino | pl.tommasino@gmail.com | 
| Paolo Zilviano | zilviano.1916518@studenti.uniroma1.it |
| | |
| | |

## 1. Data


In [1]:
import networkx as nx
import matplotlib.pyplot as plt
import ijson
import pandas as pd
from tqdm import tqdm
from collections import Counter
import random

At the first, let's see how the file is structured, let's take the first row of the dataset and see what the columns and the first row of the dataset are:

In [2]:
with open('dblp.v12.json', 'r') as file:
    # Parse the JSON array items one by one
    array_items = ijson.items(file, 'item')
    # Iterate over the JSON array items
    for item in array_items:
        dict1 = item
        break

In [3]:
#List the column of the json file
chiave = list(dict1.keys())
print(chiave)

['id', 'authors', 'title', 'year', 'n_citation', 'page_start', 'page_end', 'doc_type', 'publisher', 'volume', 'issue', 'doi', 'references', 'indexed_abstract', 'fos', 'venue']


In [4]:
#List of values of first row of the json file
valori = list(dict1.values())
print(valori)

[1091, [{'name': 'Makoto Satoh', 'org': 'Shinshu University', 'id': 2312688602}, {'name': 'Ryo Muramatsu', 'org': 'Shinshu University', 'id': 2482909946}, {'name': 'Mizue Kayama', 'org': 'Shinshu University', 'id': 2128134587}, {'name': 'Kazunori Itoh', 'org': 'Shinshu University', 'id': 2101782692}, {'name': 'Masami Hashimoto', 'org': 'Shinshu University', 'id': 2114054191}, {'name': 'Makoto Otani', 'org': 'Shinshu University', 'id': 1989208940}, {'name': 'Michio Shimizu', 'org': 'Nagano Prefectural College', 'id': 2134989941}, {'name': 'Masahiko Sugimoto', 'org': 'Takushoku University, Hokkaido Junior College', 'id': 2307479915}], 'Preliminary Design of a Network Protocol Learning Tool Based on the Comprehension of High School Students: Design by an Empirical Study Using a Simple Mind Map', 2013, 1, '89', '93', 'Conference', 'Springer, Berlin, Heidelberg', '', '', '10.1007/978-3-642-39476-8_19', [2005687710, 2018037215], {'IndexLength': 58, 'InvertedIndex': {'tool.': [42], 'study': [

### Data pre-processing

We now transform the entire *.json file* into a dictionary with selected columns (*id, authors, title, year, n_citation, doc_type, publisher* and *references*)

In [5]:
#Create a dict for Collaboration Graph
dict_entire = {
    'id' : [], 
    'authors' : [], 
    'title' : [], 
    'year' : [], 
    'n_citation' : [], 
    'doc_type' : [], 
    'publisher' : [],
    'references' : []
}

We create functions for processing the *.json file* (they are in **function.py**).

In [6]:
from function import *

with open('dblp.v12.json', 'r') as file:
    # Parse the JSON array items one by one
    array_items = ijson.items(file, 'item')
    # Iterate over the JSON array items
    for item in tqdm(array_items):
        # Process each JSON array item as needed
        dict_id(dict_entire, item)
        dict_authors(dict_entire, item)
        dict_title(dict_entire, item)
        dict_year(dict_entire, item)
        dict_n_citation(dict_entire, item)
        dict_doc_type(dict_entire, item)
        dict_publisher(dict_entire, item)
        dict_references(dict_entire, item)

0it [00:00, ?it/s]

4894081it [05:59, 13614.32it/s]


Transform dictionary in Pandas Dataframe:

In [7]:
#Transform dict in dataframe
df_entire = pd.DataFrame(dict_entire)
df_entire.head()

,id,authors,title,year,n_citation,doc_type,publisher,references
0,1091,"[Makoto Satoh, Ryo Muramatsu, Mizue Kayama, Ka...",Preliminary Design of a Network Protocol Learn...,2013,1,Conference,"Springer, Berlin, Heidelberg","[2005687710, 2018037215]"
1,1388,[Pranava K. Jha],Further Results on Independence in Direct-Prod...,2000,1,Journal,None,None
2,1674,"[G. Beale, G. Earl]",A methodology for the physically accurate visu...,2011,1,Conference,Eurographics Association,"[1535888970, 1992876689, 1993710814, 203565334..."
3,1688,"[Altaf Hossain, Faisal Zaman, M. Nasser, M. Mu...","Comparison of GARCH, Neural Network and Suppor...",2009,6,Conference,"Springer, Berlin, Heidelberg","[1560724230, 1986968751, 2156909104]"
4,5411,"[Rafael Álvarez, Leandro Tortosa, José-Francis...",COMPARING GNG3D AND QUADRIC ERROR METRICS METH...,2009,0,Conference,None,None


In [8]:
#Save df in csv file
#df_entire.to_csv('df_entire.csv')
#Read csv df file
#df_entire = pd.read_csv('df_entire.csv', index_col=0, low_memory=False)

#Drop NaN value in the column I
#df_entire = df_entire.dropna(subset='id')

#Transform columns from decimal to integer
#df_entire['id'] = df_entire['id'].astype(int)
#df_entire['n_citation'] = df_entire['n_citation'].astype(int)
#Transform column from integer to datetime (year)
#df_entire['year'] = df_entire['year'].astype('datetime64[Y]')

#See the 'head' of dataset
#df_entire.head()


#### 1. Identify the top 10,000 papers with the highest number of citations.

In [9]:
#Create the top 10000 papers with the highest number of citations
df_10th_papers = df_entire.sort_values(by=['n_citation'], ascending=False)[:10000]

#See the first five lines
df_10th_papers.head()

,id,authors,title,year,n_citation,doc_type,publisher,references
4696136,2041404167,[C. E. Shannon],The Mathematical Theory of Communication,1949,48327,Book,None,None
4630907,1639032689,[David E. Goldberg],"Genetic algorithms in search, optimization, an...",1989,44175,Book,None,None
4092588,2912565176,[Lotfi A. Zadeh],Fuzzy sets,1996,42437,None,None,None
2937610,2151103935,[David G. Lowe],Distinctive Image Features from Scale-Invarian...,2004,35541,Journal,Kluwer Academic Publishers,"[19720318, 1541642243, 1560959218, 1676552347,..."
4088311,2911964244,[Leo Breiman],Random Forests,2001,34741,None,None,"[1507255258, 1580948147, 1605688901, 197584664..."


In [10]:
#Save df in csv file
#df_10th_papers.to_csv('df_10th_papers.csv')
#Read csv df file
#df_10th_papers = pd.read_csv('df_10th_papers.csv')

#Transform columns from decimal to integer
#df_10th_papers['id'] = df_10th_papers['id'].astype(int)
#df_10th_papers['n_citation'] = df_10th_papers['n_citation'].astype(int)
#Transform column from integer to datetime (year)
#df_10th_papers['year'] = df_10th_papers['year'].astype('datetime64[Y]')

#### 2. Then the nodes of your graphs would be as follows:

#### - **Citation graph**: you can consider each of the papers as your nodes

In [11]:
#Chech if in the column 'id' there are any NaN value in column
isthere_nan = df_entire['id'].isna().any()
isthere_nan

False

In [12]:
#Create nodes for Citation Graph
citationG_nodes = dict_entire['id']
citationG_nodes[:5]

[1091, 1388, 1674, 1688, 5411]

#### - **Collaboration graph**: the authors of these papers would be your nodes

In [13]:
#Chech if in the column 'authors' there are any NaN value in column
isthere_nan = df_entire['authors'].isna().any()
isthere_nan

True

In [14]:
df_entire = df_entire.dropna(subset='authors')

In [15]:
#Create nodes for Collaboration Graph
collaborationG_nodes_2 = df_entire['authors']

#Unzip the nested lists in one list of authors
collaborationG_nodes = []

for list_1 in collaborationG_nodes_2:
    for author in list_1:
        collaborationG_nodes.append(author)

collaborationG_nodes[:5]

['Makoto Satoh',
 'Ryo Muramatsu',
 'Mizue Kayama',
 'Kazunori Itoh',
 'Masami Hashimoto']

#### 3. For the edges of the two graphs, you would have the following cases:

#### - **Citation graph**: only consider the citation relationship between these 10,000 papers and ignore the rest.

In [16]:
#Take a subset of the 10,000 papers dataset, and transform it in a readable dataset...
##... with only two column, within one value
citationG_edges_df_2 = df_10th_papers.reset_index()[['id','references']]
citationG_edges_df_2 = citationG_edges_df_2.explode('references')
citationG_edges_df_2 = citationG_edges_df_2.dropna(subset=['references']).reset_index()[['id','references']]

#Transform dataset in a list of tuple
citationG_edges = [tuple(x) for x in citationG_edges_df_2.to_records(index=False)]
citationG_edges[:5]

[(2151103935, 19720318),
 (2151103935, 1541642243),
 (2151103935, 1560959218),
 (2151103935, 1676552347),
 (2151103935, 1681491849)]

#### - **Collaboration graph**: only consider the collaborations between the authors of these 10,000 papers and ignore the rest.

In [17]:
collaboration_dict = {
    'name1' : [],
    'name2' : []
}

In [18]:
#Take a subset of the 10,000 papers dataset, and transform it in a readable dataset...
##... with only two column, within one value
collaborationG_edges_df_2 = df_10th_papers.reset_index()[['authors']]
nestedlist_auth_collaboration = collaborationG_edges_df_2['authors'].tolist()

#Create all possible matches of collaboration for each book
collaborationG_edges = []
for authors_list in nestedlist_auth_collaboration:
    for i in range(len(authors_list)):
        for j in range(i + 1, len(authors_list)):
            collaborationG_edges.append((authors_list[i], authors_list[j]))

collaborationG_edges[:5]


[('David W. Hosmer', 'Stanley Lemeshow'),
 ('Chih-Chung Chang', 'Chih-Jen Lin'),
 ('Corinna Cortes', 'Vladimir Vapnik'),
 ('Heng Li', 'Richard Durbin'),
 ('K. Deb', 'A. Pratap')]

In [19]:
#Create a counter for couple
collaborationG_edges_counter = Counter(collaborationG_edges)

#Create a dictionary for track couple weight
collaborationG_edges_weights = {}

#Iteract edges and count from dict
for edge, count in collaborationG_edges_counter.items():
    collaborationG_edges_weights[edge] = count

    #Add weight also for inverted edge
    inverted_edge = (edge[1], edge[0])
    collaborationG_edges_weights[inverted_edge] = count

#### Now we calculate the graphs:

#### - **Citation graph**:

In [20]:
#Create a unweighted and directed graph
citation_graph = nx.DiGraph()

# Add nodes to the graph
citation_graph.add_nodes_from(citationG_nodes)
# Add edges to the graph
citation_graph.add_edges_from(citationG_edges)

#### - **Collaboration graph**:

In [21]:
#Create a weighted and undirected graph
collaboration_graph = nx.Graph()

#Add nodes to the graph
collaboration_graph.add_nodes_from(collaborationG_nodes)

#Add edges to the graph
for edge, weight in collaborationG_edges_weights.items():
    collaboration_graph.add_edge(edge[0], edge[1], weight=weight)

## 2. Controlling system


### 2.1 Backend Implementation


In [22]:
#For this section (2.1) we work with the subgraph (10,000) of the entire graph

#Shortlist for Citation Subgraph
books_red = df_10th_papers['id'].tolist()
#Subgraph
sub_citation_graph = citation_graph.subgraph(books_red)

#Create the shortlist for Collaboration Subgraph
auth_red = []
for list_2 in nestedlist_auth_collaboration:
    for auth in list_2:
        auth_red.append(auth)
#Subgraph
sub_collaboration_graph = collaboration_graph.subgraph(auth_red)

#### Functionality 1 - Graph's features

**Input**:
- The graph
- The name of the graph

**Output**:
- The number of the nodes in the graph
- The number of the edges in the graph
- The graph density
- The graph degree distribution
- The average degree of the graph
- The graph hubs (hubs are nodes having degrees more extensive than the 95th percentile of the degree distribution)
- Whether the graph is dense or sparse

In [23]:
from backend import *

name_graph = 'Sub Citation Graph'
len_nodes_citation, len_edges_citation, density_citation, degree_sequence_citation, avg_degree_citation, hubs_citation, classification_graph_citation = functionality_1(sub_citation_graph, name_graph)

In [24]:
from backend import functionality_1

name_graph = 'Sub Collaboration Graph'
len_nodes_collab, len_edges_collab, density_collab, degree_sequence_collab, avg_degree_collab, hubs_collab, classification_graph_collab = functionality_1(sub_collaboration_graph, name_graph)

#### Functionality 2 - Nodes' contribution

##### Input:
- The graph
- A node of the graph (paper/author)
- The name of the graph

##### Output:

- The centrality of the node, calculated based on the following centrality measurements:
    - Betweeness
    - PageRank
    - ClosenessCentrality
    - DegreeCentrality

In [25]:
from backend import functionality_2

#node_selected_collaboration = 2155511848

name_graph = 'Sub Citation Graph'
#node_selected = int(input("Insert the Node that the function calculated: "))
betw, pagerank, closeness, degree = functionality_2(sub_citation_graph, node_selected=2155511848, name_graph=name_graph)

#### Functionality 3 - Shortest ordered walk

##### Input:
- The graph data
- A sequence of authors_a = [a_2, ..., a_{n-1}]
- Initial node a_1 and an end node a_n
- *N*: denoting the top *N* authors whose data should be considered

##### Output:
- The shortest walk of collaborations you need to read to get from author a_1 to author a_n and the papers you need to cross to realize this walk.


Considerations: 
For this functionality, you must implement an algorithm that returns the shortest __walk__ that goes from node a\_j to a\_n, which visits **in order** the nodes in _a_. The choice of a\_j and a\_n can be made randomly (or if it improves the performance of the algorithm, you can also define it in any other way) 

__Important Notes:__
- This algorithm should be run only on the collaboration graph.
- The algorithm needs to handle the case that the graph is not connected. Thus, only some nodes in _a_ are reachable from a\_1. In such a scenario, it is enough to let the program give in the output the string "There is no such path."
- Since we are dealing with walks, you can pass on the same node _a\_i_ more than once, but you must preserve order. It means you can go back to any author node any time you want, assuming that the order in which you visit the required nodes is still the same.
- Once you completed your implementation, ask chatGPT for a different one leveraging another approach in solving the shortest path and prove whether this implementation is correct. 

PROVA

In [26]:
degrees_for_topN = dict(collaboration_graph.degree())
topN_auth_2 = sorted(degrees_for_topN.items(), key=lambda x:x[1], reverse=True)[:100]
topN_auth = [i[0] for i in topN_auth_2] #taken
Nauthors_graph = collaboration_graph.subgraph(topN_auth)

In [49]:
'''
# Dijkstra's algorithm
def NextNode(dist, visited):
    
    ---
    Found the next node to operate on
    ---

    Input:
    - dist: list of distances
    - visited: list of all nodes if the node is visited (=True), not visited (=False)
    Output:
    - next_node: next node to operate on
    

    min = float('inf')
    next_node = -1
    for i in range(len(dist)):
        if (not visited[i]) and (dist[i] < min):
            next_node = i
            min = dist[i]
    return next_node
'''

def getAdjacents(graph, actual_node, next_node):
    '''
    ---
    Found the list of nodes adjacent to the node selected
    ---

    Input:
    - graph: graph
    - node: node of graph
    Output:
    - list_nodes_adjacent: list of nodes adjacent to the node
    '''

    #list_nodes_adjacent = []
    auth = [i for i in graph[actual_node]]
    weight_auth = [graph[actual_node][i]['weight'] for i in auth]
    nod_adj = list(zip(auth, weight_auth))
    '''
    for j in range(len(graph)):
        if nod_adj[1][1] != float('inf'):
            list_nodes_adjacent.append(nod_adj[j])
            '''
    return nod_adj #list_nodes_adjacent

def Dijkstra(graph, node_start, sequence_authors):
    '''
    Input:
    - graph
    - node_start: start node to calculate minimum distances
    - authors_sequence: list of authors to consider
    Output:
    - dist: list of distances
    '''

    start = 1
    n = len(graph)

    dict_grafo = {
        'index': [],
        'name_node': [],
        'visited': [],
        'dist': []
    }

    for i in range(n):
        dict_grafo['index'].append(i)
        dict_grafo['visited'].append(False)
        dict_grafo['dist'].append(float('inf'))

    dict_grafo[dict_grafo['visited'] == node_start]

    if start == 1:
        actual = node_start
        next = sequence_authors[0]
        V = getAdjacents(graph, actual, next)
        for [z, w] in V:
            index_actual = name_node
            d = dist[0] + w
            if dist[z] > d:
                dist[z] = d
        start = 0
    else:
        for i in range(n - 1):
            #next = NextNode(dist, visited)
            next = sequence_authors[i]
            visited[i] = True
            V = getAdjacents(graph, next)
            for [z, w] in V:
                d = dist[i] + w
                if dist[z] > d:
                    dist[z] = d

    return dist

def functionality_3_prova3(graph, sequence_authors, node_first, node_last, N):
    '''
    Input:
    - graph: The graph data
    - sequence_authors: A sequence of authors_a = [a_2, ..., a_{n-1}] (put 0 for a sample)
    - node_first: Initial node a_1 (put 0 for a take the first in the sample)
    - node_last: End node a_n (put 0 for a take the last in the sample)
    - N: denoting the top N authors whose data should be considered

    Output:
    - shortest_walk: The shortest walk of collaborations you need to read to get from author a_1 to author a_n
    - papers: The papers you need to cross to realize this walk.
    '''

    #Create a sub-graph from N (the top N authors with the highest degree)
    degrees_for_topN = dict(graph.degree())
    topN_auth_2 = sorted(degrees_for_topN.items(), key=lambda x:x[1], reverse=True)[:N]
    topN_auth = [i[0] for i in topN_auth_2] #taken
    Nauthors_graph = graph.subgraph(topN_auth)

    if sequence_authors==0:
        #Extract all the nodes a take a sample from this
        n = 10
        all_authors = list(Nauthors_graph.nodes)
        sequence_authors = random.sample(all_authors, n)
    
    if node_first==0:
        #Select the first...
        node_first = sequence_authors[0]
        #... and delete it from sequence authors
        sequence_authors.remove(node_first)

    if node_last==0:
        #Select the last...
        node_last = sequence_authors[-1]
        #... and delete it from sequence authors
        sequence_authors.remove(node_last)

    # final path
    shortest_walk = [].append(node_first)

    # shortest path between starting node and first node in the sequence
    path = Dijkstra(Nauthors_graph, node_first, sequence_authors)
    # add to total path (without repeating starting node)
    shortest_walk.extend(path[1:])

    # for each node in the sequence
    # find shortest path between that node and the following node in the sequence
    for p in range((len(sequence_authors)-1)):
      path = Dijkstra(Nauthors_graph, sequence_authors[p], sequence_authors[p+1])[0] 
      # add path to total_path
      shortest_walk.extend(path[1:])

    # shortest path between last node in the sequence and ending node
    path = Dijkstra(Nauthors_graph, sequence_authors[-1], node_last)[0]
    shortest_walk.extend(path[1:])

    return shortest_walk

#from backend import functionality_3_prova

shortest_walk = functionality_3_prova3(collaboration_graph, sequence_authors=0, node_first=0, node_last=0, N=100)
shortest_walk

TypeError: list indices must be integers or slices, not str

In [ ]:
#from backend import functionality_3_prova

shortest_walk = functionality_3_prova3(collaboration_graph, sequence_authors=0, node_first=0, node_last=0, N=100)
shortest_walk

VERO

In [ ]:
from backend import functionality_3

#We selected N = 1000
N_sel = 100

#shortest_walk, papers = functionality_3(collaboration_graph, sequence_authors=0, node_first=0, node_last=0, N=N_sel)


#### Functionality 4 - Disconnecting Graphs

##### Input:

- The graph data
- authorA: a paper to which will relate sub-graph G_a
- authorB: a paper to which will relate sub-graph G_b
- *N*: denoting the top *N* authors that their data should be considered

##### Output:

- The minimum number of edges (by considering their weights) required to disconnect the original graph in two disconnected subgraphs: G_a and G_b.

In [ ]:
#from backend import functionality_4

#### Functionality 5 - Extracting Communities

##### Input:

- The graph data
- *N*: denoting the top *N* papers that their data should be considered
- Paper_1: denoting the name of one of the papers
- Paper_2: denoting the name of one of the papers

##### Output:

- The minimum number of edges that should be removed to form communities
- A list of communities, each containing a list of papers that belong to them.
- Whether the Paper_1 and Paper_2 belongs to the same community.

In [ ]:
from backend import functionality_5

### 2.2. Frontend Implementation

## 3. Bonus - PageRank on MapReduce

## 4. Command Line Question (CLQ)


## 5. Algorithmic Question (AQ)